# Análise exploratória

**TODO:**
- (1) Tabela
- (2) Gráfico
- (3) Tabela

## Análise das viagens da amostra x apuradas

- Tabela de viagens identificadas da amostra: `output/analise_amostra_pre_solucao.csv`

> data	servico	sentido	id_veiculo	datetime_partida_amostra	datetime_chegada_amostra	datetime_partida_apuracao	datetime_chegada_apuracao	status

In [1]:
  #!pip install matplotlib
import basedosdados as bd
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import timedelta, datetime

pd.set_option('display.max_rows', 25)

# Paths
import os
import sys
from pathlib import Path 

current_path = Path().resolve().parent

paths = dict()
paths["raw"] = current_path / 'data' / 'raw'
paths["treated"] = current_path / 'data' / 'treated'
paths["output"] = current_path / 'data' / 'output'
paths["figures"] = current_path / 'data' / 'figures'
paths["scripts"] = current_path / 'scripts'


for path in paths.values():
    if not os.path.exists(path):
        os.makedirs(path)      

if paths["scripts"] not in sys.path:
    sys.path.append(str(paths["scripts"]))
 
from graphs import *
from categorize_trips import *

# Seta o projeto a ser cobrado
bd.config.billing_project_id = "rj-smtr-dev"

# Seta as configurações a serem usadas
bd.config.project_config_path = "C:/Users/igorl/.basedosdados/"
# bd.config.project_config_path = "D:\\basedosdados\\dev"

In [2]:
servico = "010"

#### 1 - Amostra

In [3]:
amostra = pd.read_excel('../data/raw/amostra_linha_010_q1_2023_tratado.xlsx', sheet_name='gabarito')

# Convertendo as colunas para datetime
amostra['datetime_partida'] = pd.to_datetime(amostra['datetime_partida'])
amostra['datetime_chegada'] = pd.to_datetime(amostra['datetime_chegada'])

# Criando as colunas hora_inicio e hora_fim no formato hh:mm
amostra['hora_inicio'] = amostra['datetime_partida'].dt.strftime('%H:%M')
amostra['hora_fim'] = amostra['datetime_chegada'].dt.strftime('%H:%M')

# Removendo as colunas originais
amostra = amostra.drop(columns=['extensao'])


amostra['servico'] = amostra['servico'].astype(str)
amostra['servico'] = amostra['servico'].apply(lambda x: "010" if x == "10" else x)
amostra['data'] = amostra['data'].astype(str)
amostra['hora_inicio'] = amostra['hora_inicio'].astype(str)
amostra['hora_fim'] = amostra['hora_fim'].astype(str)
amostra_tratada = amostra
amostra_tratada

,data,servico,sentido,id_veiculo,datetime_partida,datetime_chegada,hora_inicio,hora_fim
0,2023-01-02,010,I,A72037,2023-01-02 06:07:00,2023-01-02 06:22:00,06:07,06:22
1,2023-01-02,010,V,A72037,2023-01-02 06:29:00,2023-01-02 06:56:00,06:29,06:56
2,2023-01-02,010,I,A72085,2023-01-02 06:33:00,2023-01-02 06:52:00,06:33,06:52
3,2023-01-02,010,I,A72037,2023-01-02 06:58:00,2023-01-02 07:16:00,06:58,07:16
4,2023-01-02,010,V,A72085,2023-01-02 07:03:00,2023-01-02 07:29:00,07:03,07:29
...,...,...,...,...,...,...,...,...
702,2023-01-15,010,I,A72085,2023-01-15 18:51:00,2023-01-15 19:14:00,18:51,19:14
703,2023-01-15,010,V,A72085,2023-01-15 19:16:00,2023-01-15 19:38:00,19:16,19:38
704,2023-01-15,010,I,A72085,2023-01-15 19:41:00,2023-01-15 19:59:00,19:41,19:59
705,2023-01-15,010,V,A72085,2023-01-15 20:06:00,2023-01-15 20:28:00,20:06,20:28


In [4]:
datas = amostra_tratada['data'].unique()
veiculos = amostra_tratada['id_veiculo'].unique()

print("A quantidade total de viagens na amostra é:", amostra_tratada.shape[0])
print("A amostra é referente aos dias:", datas)
print("A amostra tem dados dos seguintes veículos:", veiculos) 

A quantidade total de viagens na amostra é: 707
A amostra é referente aos dias: ['2023-01-02' '2023-01-03' '2023-01-04' '2023-01-05' '2023-01-06'
 '2023-01-07' '2023-01-08' '2023-01-09' '2023-01-10' '2023-01-11'
 '2023-01-12' '2023-01-13' '2023-01-14' '2023-01-15']
A amostra tem dados dos seguintes veículos: ['A72037' 'A72085' 'A72151' 'A72048']


#### 2 - Remover viagens sobrepostas da amostra

Esta etapa classifica como "Viagem inválida - sobreposição de viagem" aquelas viagens em que um mesmo `id_veiculo` realiza múltiplas viagens em um mesmo intervalo de tempo (`datetime_partida` e `datetime_chegada`).

No caso de duas ou mais viagens sobrepostas, apenas uma será classificada nas demais etapas de classificação do notebook.

In [5]:
amostra_deduplicada = remove_overlapping_trips(amostra_tratada) 
amostra_deduplicada

,data,servico,sentido,id_veiculo,datetime_partida,datetime_chegada,hora_inicio,hora_fim,status
0,2023-01-02,010,I,A72037,2023-01-02 06:07:00,2023-01-02 06:22:00,06:07,06:22,NaN
1,2023-01-02,010,V,A72037,2023-01-02 06:29:00,2023-01-02 06:56:00,06:29,06:56,NaN
2,2023-01-02,010,I,A72085,2023-01-02 06:33:00,2023-01-02 06:52:00,06:33,06:52,NaN
3,2023-01-02,010,I,A72037,2023-01-02 06:58:00,2023-01-02 07:16:00,06:58,07:16,NaN
4,2023-01-02,010,V,A72085,2023-01-02 07:03:00,2023-01-02 07:29:00,07:03,07:29,NaN
...,...,...,...,...,...,...,...,...,...
702,2023-01-15,010,I,A72085,2023-01-15 18:51:00,2023-01-15 19:14:00,18:51,19:14,NaN
703,2023-01-15,010,V,A72085,2023-01-15 19:16:00,2023-01-15 19:38:00,19:16,19:38,NaN
704,2023-01-15,010,I,A72085,2023-01-15 19:41:00,2023-01-15 19:59:00,19:41,19:59,NaN
705,2023-01-15,010,V,A72085,2023-01-15 20:06:00,2023-01-15 20:28:00,20:06,20:28,NaN


In [6]:
count_non_nan = amostra_deduplicada['status'].notna().sum()
count_non_nan

1

In [7]:
non_nan_status_rows = amostra_deduplicada[amostra_deduplicada['status'].notna()]
non_nan_status_rows

,data,servico,sentido,id_veiculo,datetime_partida,datetime_chegada,hora_inicio,hora_fim,status
108,2023-01-03,010,I,A72085,2023-01-03 21:20:00,2023-01-03 21:36:00,21:20,21:36,Viagem duplicada na amostra


### 3 Comparar dados da amostra com os dados das viagens completas

#### 3.1 Consultar se as viagens da amostra foram feitas pelos veículos indicados na amostra

In [8]:
# separar as datas da amostra para fazer a query
datas = amostra_deduplicada['data'].unique()

q = f"""
       SELECT
         id_veiculo, servico_informado, sentido, datetime_partida, datetime_chegada
       FROM
         `rj-smtr.projeto_subsidio_sppo.viagem_completa`
       WHERE
         data IN {tuple(datas)}
         AND servico_informado = '{servico}'
       """
       
viagem_completa = bd.read_sql(q, from_file=True)
viagem_completa.info() 

viagem_completa    

Downloading: 100%|██████████| 269/269 [00:00<00:00, 709.77rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269 entries, 0 to 268
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id_veiculo         269 non-null    object        
 1   servico_informado  269 non-null    object        
 2   sentido            269 non-null    object        
 3   datetime_partida   269 non-null    datetime64[ns]
 4   datetime_chegada   269 non-null    datetime64[ns]
dtypes: datetime64[ns](2), object(3)
memory usage: 10.6+ KB


,id_veiculo,servico_informado,sentido,datetime_partida,datetime_chegada
0,A72085,010,C,2023-01-03 16:18:27,2023-01-03 17:11:18
1,A72037,010,C,2023-01-03 12:37:43,2023-01-03 13:20:33
2,A72037,010,C,2023-01-03 06:33:31,2023-01-03 07:14:08
3,A72037,010,C,2023-01-03 14:38:02,2023-01-03 15:30:13
4,A72037,010,C,2023-01-03 11:38:31,2023-01-03 12:21:44
...,...,...,...,...,...
264,A72048,010,C,2023-01-12 09:44:27,2023-01-12 10:36:09
265,A72085,010,C,2023-01-12 10:22:25,2023-01-12 11:07:26
266,A72085,010,C,2023-01-12 09:15:02,2023-01-12 10:11:44
267,A72048,010,C,2023-01-12 19:16:08,2023-01-12 20:02:18


In [9]:
viagem_completa['datetime_partida'] = pd.to_datetime(viagem_completa['datetime_partida'])

amostra_classificada = check_trips(amostra_deduplicada, viagem_completa, 10,
                                   "Viagem circular identificada e já paga")
amostra_classificada


Casos duplicados encontrados no cruzamento de dados:
         data servico_amostra sentido_amostra id_veiculo_amostra  \
0  2023-01-15             010               V             A72085   
1  2023-01-15             010               I             A72085   

  datetime_partida_amostra datetime_chegada_amostra hora_inicio hora_fim  \
0      2023-01-15 06:52:00      2023-01-15 07:04:00       06:52    07:04   
1      2023-01-15 07:05:00      2023-01-15 07:19:00       07:05    07:19   

                                   status id_veiculo_apurado servico_apurado  \
0  Viagem circular identificada e já paga             A72085             010   
1  Viagem circular identificada e já paga             A72085             010   

  sentido_apurado datetime_partida_apurado datetime_chegada_apurado  
0               C      2023-01-15 06:55:14      2023-01-15 07:17:15  
1               C      2023-01-15 06:55:14      2023-01-15 07:17:15  


,data,servico_amostra,sentido_amostra,id_veiculo_amostra,datetime_partida_amostra,datetime_chegada_amostra,hora_inicio,hora_fim,status,id_veiculo_apurado,servico_apurado,sentido_apurado,datetime_partida_apurado,datetime_chegada_apurado
0,2023-01-03,010,I,A72085,2023-01-03 21:20:00,2023-01-03 21:36:00,21:20,21:36,Viagem duplicada na amostra,NaN,NaN,NaN,NaT,NaT
1,2023-01-02,010,I,A72037,2023-01-02 06:07:00,2023-01-02 06:22:00,06:07,06:22,NaN,NaN,NaN,NaN,NaT,NaT
2,2023-01-02,010,V,A72037,2023-01-02 06:29:00,2023-01-02 06:56:00,06:29,06:56,NaN,NaN,NaN,NaN,NaT,NaT
3,2023-01-02,010,I,A72085,2023-01-02 06:33:00,2023-01-02 06:52:00,06:33,06:52,NaN,NaN,NaN,NaN,NaT,NaT
4,2023-01-02,010,I,A72037,2023-01-02 06:58:00,2023-01-02 07:16:00,06:58,07:16,NaN,NaN,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,2023-01-13,010,V,A72048,2023-01-13 16:56:00,2023-01-13 17:24:00,16:56,17:24,Viagem circular identificada e já paga,A72048,010,C,2023-01-13 17:00:16,2023-01-13 17:54:18
703,2023-01-13,010,V,A72048,2023-01-13 18:00:00,2023-01-13 18:22:00,18:00,18:22,Viagem circular identificada e já paga,A72048,010,C,2023-01-13 18:02:18,2023-01-13 19:00:08
704,2023-01-13,010,V,A72048,2023-01-13 19:04:00,2023-01-13 19:30:00,19:04,19:30,Viagem circular identificada e já paga,A72048,010,C,2023-01-13 19:08:08,2023-01-13 19:52:06
705,2023-01-13,010,V,A72048,2023-01-13 19:55:00,2023-01-13 20:19:00,19:55,20:19,Viagem circular identificada e já paga,A72048,010,C,2023-01-13 19:59:07,2023-01-13 20:40:02


In [10]:
# Corrigir linha duplicada
condition = (
    (amostra_classificada['id_veiculo_amostra'] == 'A72085') & 
    (amostra_classificada['datetime_partida_amostra'] == "2023-01-15 07:05:00")
)

# Update the columns from 'status' till the end for those rows
amostra_classificada.loc[condition, 'status':] = np.nan

In [11]:
count_non_nan = amostra_classificada['status'].notna().sum()
count_non_nan

268

### 4 Comparar dados da amostra com os dados das viagens conformidade

In [12]:
q = f"""
  SELECT
    id_veiculo, servico_informado, sentido, datetime_partida, datetime_chegada
  FROM
    `rj-smtr.projeto_subsidio_sppo.viagem_conformidade`
  WHERE
    data IN {tuple(datas)}
    AND servico_informado = '{servico}'
"""
viagem_conformidade = bd.read_sql(q, from_file=True)
viagem_conformidade.info()   
# viagem_conformidade['data'] = pd.to_datetime(viagem_conformidade['data'])

Downloading: 100%|██████████| 283/283 [00:00<00:00, 792.69rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283 entries, 0 to 282
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id_veiculo         283 non-null    object        
 1   servico_informado  283 non-null    object        
 2   sentido            283 non-null    object        
 3   datetime_partida   283 non-null    datetime64[ns]
 4   datetime_chegada   283 non-null    datetime64[ns]
dtypes: datetime64[ns](2), object(3)
memory usage: 11.2+ KB


In [13]:
amostra_classificada_conformidade = check_trips(amostra_classificada, viagem_conformidade, 10, 
                                                  "Viagem circular inválida - não atendeu o percentual de conformidade do GPS ou do trajeto")

Não existem casos duplicados no cruzamento de dados.


In [14]:
count_non_nan = amostra_classificada_conformidade['status'].notna().sum()
count_non_nan

328

Checar a quais viagens as meias viagens circulares pertencem:

In [15]:

def check_circular_trip(viagens_circulares_sem_status, viagem_completa, viagem_conformidade):
    # Certifique-se de que as colunas datetime estão em formato datetime
    viagens_circulares_sem_status['datetime_partida_amostra'] = pd.to_datetime(viagens_circulares_sem_status['datetime_partida_amostra'])
    viagens_circulares_sem_status['id_veiculo_amostra'] = viagens_circulares_sem_status['id_veiculo_amostra'].astype(str)
    
    viagem_completa['datetime_partida'] = pd.to_datetime(viagem_completa['datetime_partida'])
    viagem_completa['datetime_chegada'] = pd.to_datetime(viagem_completa['datetime_chegada'])
    
    viagem_conformidade['datetime_partida'] = pd.to_datetime(viagem_conformidade['datetime_partida'])
    viagem_conformidade['datetime_chegada'] = pd.to_datetime(viagem_conformidade['datetime_chegada'])        
    
    
    for index, row in viagens_circulares_sem_status.iterrows():
        # Primeira verificação com viagem_completa
        mask_completa = (
            (viagem_completa['id_veiculo'] == row['id_veiculo_amostra']) &
            (viagem_completa['datetime_partida'] <= row['datetime_partida_amostra']) & 
            (viagem_completa['datetime_chegada'] >= row['datetime_partida_amostra'])
        )

        # Se existir uma linha correspondente, atualize o status e copie os valores, depois continue para a próxima iteração
        if viagem_completa[mask_completa].shape[0] > 0:
            viagens_circulares_sem_status.loc[index, 'status'] = "Viagem circular identificada e já paga"
            matching_row = viagem_completa[mask_completa].iloc[0]
            viagens_circulares_sem_status.loc[index, ['id_veiculo_apurado', 'servico_apurado', 
                                            'sentido_apurado', 'datetime_partida_apurado', 
                                            'datetime_chegada_apurado']] = matching_row[['id_veiculo', 
                                                                                        'servico', 'sentido', 
                                                                                        'datetime_partida', 
                                                                                        'datetime_chegada']].values
            
            
            continue
        
        # Segunda verificação com viagem_conformidade se a primeira falhar
        mask_conformidade = (
            (viagem_conformidade['id_veiculo'] == row['id_veiculo_amostra']) &
            (viagem_conformidade['datetime_partida'] <= row['datetime_partida_amostra']) & 
            (viagem_conformidade['datetime_chegada'] >= row['datetime_partida_amostra'])
        )
        
        # Se existir uma linha correspondente, atualize o status e copie os valores
        if viagem_conformidade[mask_conformidade].shape[0] > 0:
            viagens_circulares_sem_status.loc[index, 'status'] = "Viagem circular inválida - não atendeu o percentual de conformidade do GPS ou do trajeto"
            matching_row = viagem_conformidade[mask_conformidade].iloc[0]
            viagens_circulares_sem_status.loc[index, ['id_veiculo_apurado', 'servico_apurado', 
                                            'sentido_apurado', 'datetime_partida_apurado', 
                                            'datetime_chegada_apurado']] = matching_row[['id_veiculo', 
                                                                                        'servico', 'sentido', 
                                                                                        'datetime_partida', 
                                                                                        'datetime_chegada']].values
            
            
    return viagens_circulares_sem_status


In [16]:
amostra_classificada_conformidade = check_circular_trip(amostra_classificada_conformidade,viagem_completa,viagem_conformidade)

#### 3.3 Verificar dados de GPS (Tabela `gps_sppo`)

In [17]:
amostra_sem_viagem = amostra_classificada_conformidade[amostra_classificada_conformidade['status'].isna()]
amostra_sem_viagem

data_n_encontradas_gps = amostra_sem_viagem['data'].unique()
data_n_encontradas_gps = pd.to_datetime(pd.Series(data_n_encontradas_gps))
data_n_encontradas_gps = data_n_encontradas_gps.dt.strftime('%Y-%m-%d')
data_n_encontradas_gps = data_n_encontradas_gps.values
data_n_encontradas_gps

veiculos_n_encontrados_gps = amostra_sem_viagem['id_veiculo_amostra'].unique()
veiculos_n_encontrados_gps 



# q = f"""
#   SELECT
#   id_veiculo,
#   servico,
#   timestamp_gps,
#   ST_GEOGPOINT(longitude, latitude) as posicao_veiculo_geo
# FROM
#   `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
# WHERE
#   DATA IN {tuple(data_n_encontradas_gps)}
#   AND id_veiculo IN {tuple(veiculos_n_encontrados_gps)}
# """
          
# dados_gps = bd.read_sql(q, from_file=True)
# dados_gps.info()
# dados_gps.to_csv('./../data/treated/gps_data_010.csv', index = False)

dados_gps = pd.read_csv('./../data/treated/gps_data_010.csv')


dados_gps['timestamp_gps'] = pd.to_datetime(dados_gps['timestamp_gps'])
dados_gps['servico'] = dados_gps['servico'].astype(str)
dados_gps['servico'] = dados_gps['servico'].apply(lambda x: "010" if x == "10" else x)
dados_gps 

,id_veiculo,servico,timestamp_gps,posicao_veiculo_geo
0,A72037,010,2023-01-07 17:21:39,POINT(-43.17773 -22.90645)
1,A72037,010,2023-01-07 15:33:01,POINT(-43.17425 -22.90895)
2,A72037,010,2023-01-07 10:51:31,POINT(-43.17879 -22.90103)
3,A72037,010,2023-01-07 14:16:56,POINT(-43.19135 -22.90432)
4,A72037,010,2023-01-07 15:41:22,POINT(-43.19012 -22.91386)
...,...,...,...,...
121765,A72085,010,2023-01-15 13:25:32,POINT(-43.18768 -22.91502)
121766,A72085,010,2023-01-15 10:05:57,POINT(-43.18376 -22.90373)
121767,A72085,010,2023-01-15 18:42:02,POINT(-43.19314 -22.90523)
121768,A72085,010,2023-01-15 13:52:54,POINT(-43.19288 -22.90551)


In [18]:
def set_status(row, df_check):
    # Filter the df_check by vehicle ID and time range
    filtered_df = df_check[
        (df_check['id_veiculo'] == row['id_veiculo_amostra']) & 
        (df_check['timestamp_gps'] >= row['datetime_partida_amostra']) & 
        (df_check['timestamp_gps'] <= row['datetime_chegada_amostra'])
    ]
    
    # If there are rows in the filtered dataframe, check the condition
    if not filtered_df.empty:
        if (filtered_df.iloc[0]['servico'] == row['servico_amostra']) and np.isnan(row['status']):
            return "Viagem circular inválida - sem sinal inicial/final dentro do raio de 500m"
        else:
            return "Sinal de GPS não encontrado para o veículo no horário da viagem"
            
    return row['status']

amostra_sem_viagem['status'] = amostra_sem_viagem.apply(lambda row: set_status(row, dados_gps), axis=1)

amostra_sem_viagem

,data,servico_amostra,sentido_amostra,id_veiculo_amostra,datetime_partida_amostra,datetime_chegada_amostra,hora_inicio,hora_fim,status,id_veiculo_apurado,servico_apurado,sentido_apurado,datetime_partida_apurado,datetime_chegada_apurado
268,2023-01-02,010,I,A72037,2023-01-02 06:07:00,2023-01-02 06:22:00,06:07,06:22,Viagem circular inválida - sem sinal inicial/f...,NaN,NaN,NaN,NaT,NaT
269,2023-01-02,010,I,A72085,2023-01-02 06:33:00,2023-01-02 06:52:00,06:33,06:52,Viagem circular inválida - sem sinal inicial/f...,NaN,NaN,NaN,NaT,NaT
297,2023-01-03,010,I,A72037,2023-01-03 06:03:00,2023-01-03 06:20:00,06:03,06:20,Viagem circular inválida - sem sinal inicial/f...,NaN,NaN,NaN,NaT,NaT
298,2023-01-03,010,I,A72085,2023-01-03 06:32:00,2023-01-03 06:52:00,06:32,06:52,Viagem circular inválida - sem sinal inicial/f...,NaN,NaN,NaN,NaT,NaT
318,2023-01-03,010,V,A72037,2023-01-03 16:36:00,2023-01-03 17:09:00,16:36,17:09,Viagem circular inválida - sem sinal inicial/f...,NaN,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630,2023-01-15,010,I,A72085,2023-01-15 08:01:00,2023-01-15 08:17:00,08:01,08:17,Viagem circular inválida - sem sinal inicial/f...,NaN,NaN,NaN,NaT,NaT
631,2023-01-15,010,V,A72085,2023-01-15 08:27:00,2023-01-15 09:02:00,08:27,09:02,Viagem circular inválida - sem sinal inicial/f...,NaN,NaN,NaN,NaT,NaT
632,2023-01-15,010,I,A72085,2023-01-15 09:06:00,2023-01-15 09:24:00,09:06,09:24,Viagem circular inválida - sem sinal inicial/f...,NaN,NaN,NaN,NaT,NaT
634,2023-01-15,010,V,A72085,2023-01-15 10:25:00,2023-01-15 10:49:00,10:25,10:49,Viagem circular inválida - sem sinal inicial/f...,NaN,NaN,NaN,NaT,NaT


In [19]:
amostra_com_viagem = amostra_classificada_conformidade[~amostra_classificada_conformidade['status'].isna()]
# juntar as duas 
result = pd.concat([amostra_com_viagem, amostra_sem_viagem], ignore_index=True)

# Ajuste na classificação da linha duplicada
condition = (result['datetime_partida_amostra'] == "2023-01-03 21:20:00") & (result['id_veiculo_amostra'] == "A72085")
result.loc[condition, 'status'] = "Viagem duplicada na amostra"

# Ajuste na classificação da linha em que não tem dados de GPS
condition = (result['datetime_partida_amostra'] == "2023-01-04 06:00:00") & (result['id_veiculo_amostra'] == "A72037")
result.loc[condition, 'status'] = "Sinal de GPS não encontrado para o veículo no horário da viagem"

result.to_excel('.././data/output/analise_amostra_pre_solucao.xlsx')

In [20]:
valor_por_categoria = result['status'].value_counts(dropna=False)
valor_por_categoria
# esse nan foi um caso que não tinha dado de GPS

Viagem circular identificada e já paga                                                      531
Viagem circular inválida - não atendeu o percentual de conformidade do GPS ou do trajeto    119
Viagem circular inválida - sem sinal inicial/final dentro do raio de 500m                    52
Sinal de GPS não encontrado para o veículo no horário da viagem                               4
Viagem duplicada na amostra                                                                   1
Name: status, dtype: int64

### Exemplos (mapa)

- Mapa de 1 viagem identificada e 1 não ientificada

## Análise histórica da linha

- Gráfico POD até a data mais recente (quando fazemos a avaliação): `analise_pod_historico.png`

In [21]:
# POD pré-reprocessamento
q = f"""
SELECT
  data, viagens, perc_km_planejada
FROM
  `rj-smtr.dashboard_subsidio_sppo.sumario_servico_dia_historico`
WHERE
  servico = '010'
  AND DATA BETWEEN '2022-06-01' AND '2023-09-30'
"""
pod_pre_reprocessamento = bd.read_sql(q, from_file=True)

Downloading: 100%|██████████| 455/455 [00:00<00:00, 1318.85rows/s]


In [22]:
# Assegurando que a coluna 'data' seja do tipo datetime
pod_pre_reprocessamento = pod_pre_reprocessamento.sort_values('data')
pod_pre_reprocessamento['data'] = pd.to_datetime(pod_pre_reprocessamento['data'])
# Criando a nova coluna 'mes_ano'
pod_pre_reprocessamento['mes_ano'] = pod_pre_reprocessamento['data'].dt.to_period('M')
pod_pre_reprocessamento['mes_ano'] = pod_pre_reprocessamento['mes_ano'].astype(str)
pod_pre_reprocessamento

,data,viagens,perc_km_planejada,mes_ano
241,2022-07-01,0,0.00,2022-07
243,2022-07-02,0,0.00,2022-07
334,2022-07-03,0,0.00,2022-07
414,2022-07-04,0,0.00,2022-07
402,2022-07-05,0,0.00,2022-07
...,...,...,...,...
450,2023-09-24,33,117.65,2023-09
200,2023-09-25,64,106.67,2023-09
438,2023-09-26,64,106.67,2023-09
57,2023-09-27,64,106.67,2023-09


In [23]:
# Calculando a mediana, Q1 e Q3 para pod_pre_reprocessamento
median_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].median()
q1_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].quantile(0.25)
q3_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].quantile(0.75)

In [24]:
# Criar o gráfico de linha
fig = go.Figure()

# Adicionando área sombreada para pod_pre_reprocessamento
fig.add_trace(go.Scatter(x=median_pre.index, y=q1_pre.values, 
                         line=dict(width=0), fill=None, mode='lines', name='Q1 Pre'))
fig.add_trace(go.Scatter(x=median_pre.index, y=q3_pre.values, 
                         fill='tonexty', fillcolor='rgba(30,144,255,0.3)', mode='lines', name='Q3 Pre', 
                         line=dict(width=0))) 

# Adicionando a linha da mediana para pod_pre_reprocessamento
fig.add_trace(go.Scatter(x=median_pre.index, y=median_pre.values,
                    mode='lines+markers',
                    name='Mediana Antes do Reprocessamento',
                    marker=dict(color='dodgerblue'),
                    opacity=0.75))

# Configurar layout e outros parâmetros do gráfico
fig.update_layout(
    title=dict(text="Percentual de Operação Diária (POD) por mês - Serviço: 010", font=dict(color='black')),
    showlegend=False,
    yaxis_range=[0, 150],
    yaxis_ticksuffix="%",
    width=800,
    height=600,
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    # Adicionando anotações para o subtítulo com cores específicas
    annotations=[
        dict(
            x=-0.065,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='Mediana, 1º e 3º quartis mensais do POD',
            showarrow=False,
            font=dict(color='black', size=14)  # Aplicando a cor do título
        ),
        dict(
            x=0.40,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='antes do reprocessamento',
            showarrow=False,
            font=dict(color='dodgerblue', size=14)
        )
    ]
)

# Adicionar linha horizontal para indicar o mínimo de 80%
fig.add_hline(y=80, annotation_text="min = 80%")

fig.show()

### Exemplos (mapa)

Viagem correta

In [25]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2022-12-29"
  AND servico = '010'
"""
       
shape = bd.read_sql(q, from_file=True)
shape.info() 

Downloading: 100%|██████████| 2/2 [00:00<00:00,  6.69rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [26]:
q = f"""
SELECT
  id_veiculo,
  servico_informado as servico,
  timestamp_gps,
  posicao_veiculo_geo,
  status_viagem
FROM
  `rj-smtr.projeto_subsidio_sppo.registros_status_viagem`
WHERE
  DATA = "2022-12-29"
  AND servico_informado = '010'
  AND timestamp_gps BETWEEN "2022-12-29 20:40:51"
  AND "2022-12-29 21:24:53"
  AND id_veiculo = "A72085"
"""

gps = bd.read_sql(q, from_file=True)
gps.info() 

Downloading: 100%|██████████| 91/91 [00:00<00:00, 267.64rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           91 non-null     object        
 1   servico              91 non-null     object        
 2   timestamp_gps        91 non-null     datetime64[ns]
 3   posicao_veiculo_geo  91 non-null     object        
 4   status_viagem        91 non-null     object        
dtypes: datetime64[ns](1), object(4)
memory usage: 3.7+ KB


In [27]:
create_trip_map(gps, shape)

Viagem não identificada por conta da alteração no shape:

In [28]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2023-02-01"
  AND servico = '010'
"""
       
shape = bd.read_sql(q, from_file=True)
shape.info() 

Downloading: 100%|██████████| 2/2 [00:00<00:00,  6.01rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [29]:
q = f"""
SELECT
  id_veiculo,
  servico_informado as servico,
  timestamp_gps,
  posicao_veiculo_geo,
  status_viagem
FROM
  `rj-smtr.projeto_subsidio_sppo.registros_status_viagem`
WHERE
  DATA = "2023-02-01"
  AND servico_informado = '010'
  AND timestamp_gps BETWEEN "2023-02-01 21:00:22"
  AND "2023-02-01 21:49:51"
  AND id_veiculo = "A72085"
"""

gps = bd.read_sql(q, from_file=True)
gps.info() 

Downloading: 100%|██████████| 79/79 [00:00<00:00, 220.06rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           79 non-null     object        
 1   servico              79 non-null     object        
 2   timestamp_gps        79 non-null     datetime64[ns]
 3   posicao_veiculo_geo  79 non-null     object        
 4   status_viagem        79 non-null     object        
dtypes: datetime64[ns](1), object(4)
memory usage: 3.2+ KB


In [30]:
create_trip_map(gps, shape)

## Análise das viagens da amostra x apuradas (pós-solução)

- Tabela de viagens identificadas da amostra: `output/analise_amostra_pos_solucao.csv`

> data	servico	sentido	id_veiculo	datetime_partida_amostra	datetime_chegada_amostra	datetime_partida_apuracao_solucao	datetime_chegada_apuracao_solucao	status

In [31]:
datas = amostra_tratada['data'].unique()
veiculos = amostra_tratada['id_veiculo'].unique()

q = f"""
SELECT
  id_veiculo, servico_informado, sentido, datetime_partida, datetime_chegada
FROM
  `rj-smtr-dev.SMTR202212006611_reprocessamento.viagem_completa`
WHERE
  data IN {tuple(datas)}
  AND servico_informado = '{servico}'
"""
viagem_completa_reprocessada = bd.read_sql(q, from_file=True)

Downloading: 100%|██████████| 734/734 [00:00<00:00, 1839.61rows/s]


In [32]:
viagem_completa['datetime_partida'] = pd.to_datetime(viagem_completa['datetime_partida'])

amostra_classificada_reprocessada = check_trips(amostra_deduplicada, viagem_completa_reprocessada, 10,
                                   "O veículo existe e operou na linha indicada pelo recurso")
amostra_classificada_reprocessada

Não existem casos duplicados no cruzamento de dados.


,data,servico_amostra,sentido_amostra,id_veiculo_amostra,datetime_partida_amostra,datetime_chegada_amostra,hora_inicio,hora_fim,status,id_veiculo_apurado,servico_apurado,sentido_apurado,datetime_partida_apurado,datetime_chegada_apurado
0,2023-01-03,010,I,A72085,2023-01-03 21:20:00,2023-01-03 21:36:00,21:20,21:36,Viagem duplicada na amostra,NaN,NaN,NaN,NaT,NaT
1,2023-01-03,010,I,A72151,2023-01-03 19:19:00,2023-01-03 20:17:00,19:19,20:17,NaN,NaN,NaN,NaN,NaT,NaT
2,2023-01-04,010,I,A72037,2023-01-04 06:00:00,2023-01-04 06:16:00,06:00,06:16,NaN,NaN,NaN,NaN,NaT,NaT
3,2023-01-04,010,V,A72037,2023-01-04 06:28:00,2023-01-04 07:01:00,06:28,07:01,NaN,NaN,NaN,NaN,NaT,NaT
4,2023-01-04,010,I,A72085,2023-01-04 06:30:00,2023-01-04 06:50:00,06:30,06:50,NaN,NaN,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,2023-01-13,010,I,A72048,2023-01-13 19:33:00,2023-01-13 19:53:00,19:33,19:53,O veículo existe e operou na linha indicada pe...,A72048,010,I,2023-01-13 19:34:46,2023-01-13 19:52:06
703,2023-01-13,010,V,A72048,2023-01-13 19:55:00,2023-01-13 20:19:00,19:55,20:19,O veículo existe e operou na linha indicada pe...,A72048,010,V,2023-01-13 19:58:27,2023-01-13 20:12:07
704,2023-01-13,010,I,A72048,2023-01-13 20:21:00,2023-01-13 20:43:00,20:21,20:43,O veículo existe e operou na linha indicada pe...,A72048,010,I,2023-01-13 20:23:01,2023-01-13 20:40:02
705,2023-01-13,010,V,A72048,2023-01-13 20:46:00,2023-01-13 21:09:00,20:46,21:09,O veículo existe e operou na linha indicada pe...,A72048,010,V,2023-01-13 20:50:02,2023-01-13 21:01:03


In [33]:
count_non_nan = amostra_classificada_reprocessada['status'].notna().sum()
count_non_nan

700

In [34]:
q = f"""
  SELECT
    id_veiculo, servico_informado, sentido, datetime_partida, datetime_chegada
  FROM
    `rj-smtr-dev.SMTR202212006611_reprocessamento.viagem_conformidade`
  WHERE
    data IN {tuple(datas)}
    AND servico_informado = '{servico}'
"""
viagem_conformidade_reprocessada = bd.read_sql(q, from_file=True)
viagem_conformidade_reprocessada.info()  

Downloading: 100%|██████████| 734/734 [00:00<00:00, 1882.10rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 734 entries, 0 to 733
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id_veiculo         734 non-null    object        
 1   servico_informado  734 non-null    object        
 2   sentido            734 non-null    object        
 3   datetime_partida   734 non-null    datetime64[ns]
 4   datetime_chegada   734 non-null    datetime64[ns]
dtypes: datetime64[ns](2), object(3)
memory usage: 28.8+ KB


In [35]:
amostra_conformidade_reprocessada = check_trips(amostra_classificada_reprocessada, viagem_conformidade_reprocessada, 10, 
                                                  "Viagem circular inválida - não atendeu o percentual de conformidade do GPS ou do trajeto")

Não existem casos duplicados no cruzamento de dados.


In [36]:
count_non_nan = amostra_conformidade_reprocessada['status'].notna().sum()
count_non_nan

700

In [37]:
amostra_sem_viagem = amostra_conformidade_reprocessada[amostra_conformidade_reprocessada['status'].isna()]
amostra_sem_viagem['status'] = amostra_sem_viagem.apply(lambda row: set_status(row, dados_gps), axis=1)

amostra_com_viagem = amostra_conformidade_reprocessada[~amostra_conformidade_reprocessada['status'].isna()]
result = pd.concat([amostra_com_viagem, amostra_sem_viagem], ignore_index=True)



# Ajuste na classificação da linha duplicada
condition = (result['datetime_partida_amostra'] == "2023-01-03 21:20:00") & (result['id_veiculo_amostra'] == "A72085")
result.loc[condition, 'status'] = "Viagem duplicada na amostra"

# Ajuste na classificação da linha em que não tem dados de GPS
condition = (result['datetime_partida_amostra'] == "2023-01-04 06:00:00") & (result['id_veiculo_amostra'] == "A72037")
result.loc[condition, 'status'] = "Sinal de GPS não encontrado para o veículo no horário da viagem"

result.to_excel('.././data/output/analise_amostra_pos_solucao.xlsx')


In [38]:
valor_por_categoria = result['status'].value_counts(dropna=False)
valor_por_categoria
# esse nan foi um caso que não tinha dado de GPS

O veículo existe e operou na linha indicada pelo recurso                     699
Sinal de GPS não encontrado para o veículo no horário da viagem                4
Viagem circular inválida - sem sinal inicial/final dentro do raio de 500m      3
Viagem duplicada na amostra                                                    1
Name: status, dtype: int64

### Exemplos (mapa)

- Mapa de 1 viagem identificada (que não foi iedntificada antes da
  solução): `analise_mapa_viagem_identificada_pos_solucao.html`
- Mapa de viagens não identificadas (se necessário i.e. teve sinal de
  GPS na linha no período, mas não teve a viagem): `analise_mapa_viagem_nao_identificada_pos_solucao_[descricao].html`